# SACC with clusters
The default SACC scripts in the directory above show how one can make a SACC object for a 3x2 point analysis. The constructor for a SACC object has additional fields for handling clusters. This notebook details how one can use those fields to create/load/split a SACC that has cluster information.

Note: this notebook is for *stacks* of clusters. Individual cluster measurements are not yet supported by SACC.

In [1]:
import sacc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/Users/tmcclintock/anaconda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tracers = [] # The list to hold all tracers

## Cluster stack details
Stacks of clusters are different from regular tracers, since they are binned not only in redshift but also by richness. In this example, we have 20 cluster stacks: 5 bins in richness and 4 tomographic bins. Since this is a tomographic analysis, each cluster stack can be associated with some number of source bins. This association is handled in later cells.

The following two cells create two sets of tracers:
1. cluster stack tracers, that hold tomographic and mass-proxy (aka richness) bin edges
2. source galaxy tracers, that are associated with individual $\gamma_T$ weak lensing profiles for each stack-souce tracer pair

In [3]:
# First, create the cluster stack tracers.
# Clusters are split into 5 richness bins and 4 tomographic bins:
lambda_bins = [20,30,50,80,120,180] #edges of richness bins
# Loop over centers of tomographic bins
for i,z in enumerate([0.3,0.5,0.7,0.9]):
    zar=np.arange(z-0.1,z+0.1,0.001)
    Nz=np.exp(-(z-zar)**2/(2*0.03**2))
    # Loop over richness bins
    for j in range(0,len(lambda_bins)-1):
        l_min = lambda_bins[j]
        l_max = lambda_bins[j+1]

        T=sacc.Tracer("clusters_z%i_l%i"%(i,j),"spin0",zar,Nz,exp_sample="clusters", 
                      Mproxy_name = "richness", Mproxy_min = l_min, Mproxy_max = l_max)
        tracers.append(T)

In [4]:
# Tomographic LSST source galaxies with 4 redshift bins
for i,z in enumerate([0.5,0.7,0.9,1.1]):
    zar=np.arange(z-0.1,z+0.1,0.001)
    Nz=np.exp(-(z-zar)**2/(2*0.025**2))
    DNz=np.zeros((len(Nz),2))
    ## some random shapes of Nz to marginalise over
    DNz[:,0]=(z-zar)**2*0.01
    DNz[:,0]-=DNz[:,0].mean()
    DNz[:,1]=(z-zar)**3*0.01
    DNz[:,1]-=DNz[:,1].mean()
    T=sacc.Tracer("lsst_sources_%i"%i,"spin2",zar,Nz,exp_sample="lsst_sources",
                               DNz=DNz)
    tracers.append(T)

### Data vectors and binning
The SACC holds data vectors and binning information. In this example, we have binning for cluster number counts as well as binning for cluster-source lensing profiles. Both are created in the following cell, as well as the binning information.

In [5]:
# Here we have 10 radial bins for cluster weak lensing
# Note that the "radial bins" can be actual distances or angles on the sky
rvals=np.logspace(np.log10(0.5), np.log10(3), 10)
Ntracer=len(tracers)
#These lists will hold the information to create the "binning" object, which
#contains all information for the projected bins on the sky we are
#working with.
type,bins,t1,q1,t2,q2,val,err=[],[],[],[],[],[],[],[]
#Loop over all tracers
for t1i in range(Ntracer):
    # If the tracer is a cluster stack, we are interested in
    # the number counts as an observable.
    if tracers[t1i].is_CL():
        type.append('+N') #Number counts
        bins.append(-1.)  #counts have no spatial bin value
        ## We refer to tracers by their index
        t1.append(t1i)
        t2.append(-1) #There is no paired tracer
        ## Here we have cluster number counts
        q1.append('S')
        q2.append('0')
        ## values and errors
        ## Number of clusters is the index, and the error is the sqrt.
        val.append(t1i+1)
        err.append(np.sqrt(float(t1i+1)))

    # Loop over all other tracers
    for t2i in range(t1i+1,Ntracer):
        # Clusters can pair with all the lensing tracers (i.e. the galaxy tomographic bins)
        if tracers[t1i].is_CL() and tracers[t2i].is_WL():
            # Loop over radial bins
            for r in rvals:
                ## we have configuration space measurement
                type.append('+R')
                ## at this nominal rbins
                bins.append(r)
                ## We refer to tracers by their index
                t1.append(t1i)
                t2.append(t2i)
                ## Here we have density-shear cross-correlations
                q1.append('S')
                q2.append('E')
                ## values and errors
                val.append(np.random.uniform(0,10))
                err.append(np.random.uniform(1,2))

# Create the binning object
binning=sacc.Binning(type,bins,t1,q1,t2,q2)
# As well as the mean vector object
mean=sacc.MeanVec(val)

### Covariance matrices
Finally, the SACC object holds a covariance matrix between all of the data we use. We will use ell_block_diagonal where everything is coupled across tracers/redshifts at the same ell but not across ell with fixed 10% off-diagonal elements.

In [6]:
## We need to add a covariance matrix. We will use ell_block_diagonal mode,
## even though in real space cluster lensing is NOT block diagonal.
## This is just for illustrative purposes and isn't realistic, since
## in reality a real space covariance matrix will be the 'dense' mode.
Np=binning.size()
cov=np.zeros((Np,Np))
#Loop over all observable bins
for i in range(Np):
    for j in range(i,Np):
        #If the observable bin is a number count,
        #then assume Poisson error: Var(N) = N
        if binning.binar['type'][i] == b'+N':
            if i == j:
                cov[i,i] = err[i]*err[i]
        #The observable is a real-space lensing bin.
        #If this is between two different bins, then
        #reduce the covariance by a factor of 10.
        if binning.binar['type'][i] == b'+R':
            cov[i,j] = err[i]*err[j]
            if i != j:
                cov[i,j] /= 10.
            cov[j,i] = cov[i,j]

#Create a precision matrix out of the covariance.
precision=sacc.Precision(cov, "ell_block_diagonal", is_covariance=True, binning=binning)

In [7]:
#Save the SACC
# Add some meta data
meta={"Creator":"McGyver","Project":"Victory"}

# finally, create SACC object and save
s=sacc.SACC(tracers,binning,mean,precision,meta)
s.printInfo()
s.saveToHDF ("test_clusters.sacc")

--------------------------------
Meta information:
   Creator : McGyver
   Project : Victory
--------------------------------
 EXP_SAMPLE: lsst_sources
       Tomographic sample #0: <z>=0.50
       Tomographic sample #1: <z>=0.70
       Tomographic sample #2: <z>=0.90
       Tomographic sample #3: <z>=1.10
 EXP_SAMPLE: clusters
       Tomographic sample #0: <z>=0.30
       Tomographic sample #1: <z>=0.30
       Tomographic sample #2: <z>=0.30
       Tomographic sample #3: <z>=0.30
       Tomographic sample #4: <z>=0.30
       Tomographic sample #5: <z>=0.50
       Tomographic sample #6: <z>=0.50
       Tomographic sample #7: <z>=0.50
       Tomographic sample #8: <z>=0.50
       Tomographic sample #9: <z>=0.50
       Tomographic sample #10: <z>=0.70
       Tomographic sample #11: <z>=0.70
       Tomographic sample #12: <z>=0.70
       Tomographic sample #13: <z>=0.70
       Tomographic sample #14: <z>=0.70
       Tomographic sample #15: <z>=0.90
       Tomographic sample #16: <z>=0.90


## Loading and splitting
A SACC object with cluster information can be loaded and split, just like the example SACC in the 3x2pt analysis.

In [8]:
s=sacc.SACC.loadFromHDF("test_clusters.sacc")
s.printInfo()

--------------------------------
Meta information:
   Creator : b'McGyver'
   Project : b'Victory'
--------------------------------
 EXP_SAMPLE: b'lsst_sources'
       Tomographic sample #0: <z>=0.50
       Tomographic sample #1: <z>=0.70
       Tomographic sample #2: <z>=0.90
       Tomographic sample #3: <z>=1.10
 EXP_SAMPLE: b'clusters'
       Tomographic sample #0: <z>=0.30
       Tomographic sample #1: <z>=0.30
       Tomographic sample #2: <z>=0.30
       Tomographic sample #3: <z>=0.30
       Tomographic sample #4: <z>=0.30
       Tomographic sample #5: <z>=0.50
       Tomographic sample #6: <z>=0.50
       Tomographic sample #7: <z>=0.50
       Tomographic sample #8: <z>=0.50
       Tomographic sample #9: <z>=0.50
       Tomographic sample #10: <z>=0.70
       Tomographic sample #11: <z>=0.70
       Tomographic sample #12: <z>=0.70
       Tomographic sample #13: <z>=0.70
       Tomographic sample #14: <z>=0.70
       Tomographic sample #15: <z>=0.90
       Tomographic sample #1

In [9]:
print ("Precision matrix, first 10 colums:")
## This will invert matrix from covariance on the fly
m=s.precision.precisionMatrix()
Ncol = 10
for i in range(Ncol):
    for j in range(Ncol):
        print ('%5.5f '%m[i,j],end="")
    print()

Precision matrix, first 10 colums:
1.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 
0.00000 0.39706 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 
0.00000 0.00000 0.65478 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 
0.00000 0.00000 0.00000 0.37748 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 
0.00000 0.00000 0.00000 0.00000 0.44082 0.00000 0.00000 0.00000 0.00000 0.00000 
0.00000 0.00000 0.00000 0.00000 0.00000 0.37776 0.00000 0.00000 0.00000 0.00000 
0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.27570 0.00000 0.00000 0.00000 
0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.55032 0.00000 0.00000 
0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.31635 0.00000 
0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.65148 


In [10]:
#Splitting the SACC
s.mean.saveToHDFFile("cluster_split.mean.sacc")
s.precision.saveToHDFFile("cluster_split.precision.sacc")
s.saveToHDF("cluster_split.main.sacc",save_mean=False, mean_filename="cluster_split.mean.sacc",
            save_precision=False, precision_filename="cluster_split.precision.sacc")

In [11]:
#We can now reload, and the SACC will be reconsituted from three files.
sn=sacc.SACC.loadFromHDF("cluster_split.main.sacc")
sn.printInfo()

--------------------------------
Meta information:
   Creator : b'McGyver'
   Project : b'Victory'
--------------------------------
 EXP_SAMPLE: b"b'clusters'"
       Tomographic sample #0: <z>=0.30
       Tomographic sample #1: <z>=0.30
       Tomographic sample #2: <z>=0.30
       Tomographic sample #3: <z>=0.30
       Tomographic sample #4: <z>=0.30
       Tomographic sample #5: <z>=0.50
       Tomographic sample #6: <z>=0.50
       Tomographic sample #7: <z>=0.50
       Tomographic sample #8: <z>=0.50
       Tomographic sample #9: <z>=0.50
       Tomographic sample #10: <z>=0.70
       Tomographic sample #11: <z>=0.70
       Tomographic sample #12: <z>=0.70
       Tomographic sample #13: <z>=0.70
       Tomographic sample #14: <z>=0.70
       Tomographic sample #15: <z>=0.90
       Tomographic sample #16: <z>=0.90
       Tomographic sample #17: <z>=0.90
       Tomographic sample #18: <z>=0.90
       Tomographic sample #19: <z>=0.90
 EXP_SAMPLE: b"b'lsst_sources'"
       Tomographic

In [12]:
#This is an example of how to call the sortTracers() function
#which returns information about the kinds of measurements we have
print(len(sn.tracers))
s2 = sn.sortTracers()
print(len(s2)) #Wrong. Should be 100 long
print(s2[0])
print(s2[1])

24
100
(0, -1, b'+N', None, None)
(0, 20, b'+R', array([0.5       , 0.61014247, 0.74454767, 0.9085603 , 1.1087024 ,
       1.3529329 , 1.6509637 , 2.014646  , 2.4584422 , 3.        ],
      dtype=float32), array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]))
